# Conversion to ONNX
ONNX is a different format for running machine learning models. The ONNX format is much faster on CPU, sometimes 5 times as fast as PyTorch!

While the EAWSW model is designed to be small, accurate and accessible, for some people it's still too much to run...

Hosting the model as a free service for players is an option. An ONNX version of the model allows us to host the model on CPU yet have faster response times! Given that the model is made in a time with chip shortage, running on hardware I already have inside a server is efficient, scalable and cheaper.

An important note is that ONNX doesn't execute logic by itself, and you have to do that yourself, `onnx_model_manager.py` intends to deal with this for us.

In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, split_branches, get_model, set_pretrained_model_dropout, get_dataset, ModelSeeder
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import time
import onnx
import logging
from onnx_model_manager import OnnxModelManager
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import datasets
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from model_manager import ModelManager

In [2]:
saved_model_path = os.path.join("models", "awsw_main")
saved_model_onnx_path = os.path.join("models", "awsw_onnx")
if not os.path.exists(os.path.join(saved_model_path, "special_tokens_map.json")):
    print("Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!")
    !cd $saved_model_path && git clone https://huggingface.co/EleutherAI/gpt-neo-125M
    !cp -n $saved_model_path/gpt-neo-125M/* $saved_model_path
    !rm -rf $saved_model_path/gpt-neo-125M
if not os.path.exists(os.path.join(saved_model_onnx_path, "model.onnx")):
    !python3 -m transformers.onnx --model=$saved_model_path --feature=causal-lm-with-past $saved_model_onnx_path

Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!
Cloning into 'gpt-neo-125M'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 44 (delta 20), reused 0 (delta 0)
Unpacking objects: 100% (44/44), 543.14 KiB | 1.15 MiB/s, done.
Using framework PyTorch: 1.10.1+cu113
Overriding 1 configuration item(s)
	- use_cache -> True
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTor

In [3]:
def optimize_onnx():
    model_quant = os.path.join(saved_model_onnx_path, "model_quant.onnx")
    if not os.path.exists(model_quant):
        model_fp32 = os.path.join(saved_model_onnx_path, "model.onnx")
        model_opt = os.path.join(saved_model_onnx_path, "model-opt.onnx")
        quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QInt8)
        #!rm $model_opt
optimize_onnx()

In [4]:
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

onnx_model_manager = OnnxModelManager(os.path.join(saved_model_onnx_path, "model-opt.onnx"))
onnx_model_manager_quant = OnnxModelManager(os.path.join(saved_model_onnx_path, "model_quant.onnx"))
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
model = AutoModelForCausalLM.from_pretrained(saved_model_path)
model.to(device)
model.eval()
model_manager = ModelManager(model=model, tokenizer=tokenizer, device=device)
print(f"Pretrained model loaded on {device_name}")

Pretrained model loaded on cuda:0


In [5]:
prompt = "In my dreams, I'm a dragon"
for i in range(2):
    print("ONNX:", onnx_model_manager.say_raw(prompt, do_sample=True))
    print("ONNX (Quantized):", onnx_model_manager_quant.say_raw(prompt, do_sample=True))
    print("PyTorch:", model_manager.say_raw(prompt, 50, 0.7))
    print('-' * 100)

ONNX: In my dreams, I'm a dragon."<p><msg>Ry "I see."<|endoftext|>
ONNX (Quantized): In my dreams, I'm a dragon. I even had a dream about a dragon that had a portal for all the time. I had no idea what they were going to do, and I had no idea what they were going to do. I had no idea what they were going to do, and I had no idea what they were going to do. I had no idea what they were going to do, and I had no idea what they were going to do, and I had no idea what they were going to do, and I had no idea what they were going to do, and I had no idea what they were going to do, and I had no idea
PyTorch: In my dreams, I'm a dragon."<d><scn>black<msg>An "The dragon has wings, and they fly in a wide arc, like a human. We see them in different colors, and the dragon is the strongest one."<p><msg>c "Do you want to see some of the rest?"<d><scn>black<msg>An "Oh, come on, it's pretty obvious. I don't know how you do it."<p><msg>c "Is it still kinda scary?"<d><scn>black<msg
------------------

# Testing

We created a few past (for context) + present prompts (player input) and see the different reactions. This way, we can test the models across different iterations.
The first test involves a old prompt to compare the pre-trained model with the one trained on AWSW. Did it manage to store it's data well? Is it able to write down things that have nothing to do with AWSW? (So we know we didn't overfit).

**This test generates boring and repetetive** replies! It's because we use no good sampling algorithm, but it does give us a indication of what the model has learned!

In [6]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hello, [player_name]."', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><scn>park2<msg>Ad "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
]

for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "I'm good."<|endoftext|>

[ONNX] Reply: park2<msg>Ry "I'm good."<|endoftext|>

[ONNX Quantized] Reply: <msg>c "I'm good."<p><msg>c "And a little more. I think that's all we need."<p><msg>c "If we don't get a little more, we might even end up with a whole town."<p><msg>c "meet with maverick"<d><scn>park3<msg>Mv "If we had a better plan, we could all get a little more. We could even get a little more. We could even get a little more. We could even get a little more. We could even

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad "I think he is a bit odd."<|endoftext|>

[ONNX] Reply: park2<msg>Ad "I think he is a bit odd."<|endoftext|>

[ONNX Quantized] Reply: <msg>Ad "I think we should all be happy."<d><scn>l<msg>Lo "I think we should all be happy."<p><msg>c "I think we should all be happy."<p><msg>c "I think we should all be happy."<d><scn>n?<p><msg>Lo "I think we should all be happy."<p><msg>c "I think we sh

# Sampling test

This is gonna be interesting!

In [7]:
for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt, top_k = 50, top_p = 0.7)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt, do_sample = True)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt, do_sample = True)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "Hey [player_name]!"<|endoftext|>

[ONNX] Reply: park2<msg>Ry "I'm good."<|endoftext|>

[ONNX Quantized] Reply: <msg>c "I think so. How's that work out for a little while."<p><msg>c "meet Kevin"<d><scn> metadnal<msg>Lo "K-k-o-n-o<d>n"<p><msg>c "And that's what we are all for now. You can't even start a game with me."<p><msg>c "And that's what we're going through."<p><msg>c "And that is all that we are going for."<d><msg

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad "I think he is nice."<|endoftext|>

[ONNX] Reply: park2<msg>Ad "What are you even doing here? You should get some rest."<p><msg>c "I'm not really a doctor."<d><scn>park2<msg>Ad "I don‘t know what you're talking about."<p><msg>c "I'm sorry. You're not supposed to be in the mood for a medical treatment."<d><scn>park2<msg>Ad "I don't know what you're doing, but if I were, you wouldn‘t mind if I went to the doctor."

[ONNX Quantized] Reply: <msg>A

# RP test
Testing out the injected roleplay actions

In [8]:
test_rps = [
    "Visit Lorem",
    "Meet with Lorem",
    "Visit Adine",
    "Fight Maverick",
    "Fight Adine",
    "Attack Adine"
]

for rp in test_rps:
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)
    
print("Lowercase test")

for rp in test_rps:
    rp = rp[0].lower() + rp[1:]
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    rp = rp.lower()
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)

[Pytorch] Visit Lorem -> loremapt<msg>Lo "Hey [player_name]! How are you?"<|endoftext|>
[ONNX] Visit Lorem -> loremapt<msg>Lo "Hey [reps]! Over here"<|endoftext|>
[ONNX Quantized] Visit Lorem -> <msg>c "Andrea"<d><scn>lorem-ad "adineum",<p><|endoftext|>
----------
[Pytorch] Meet with Lorem -> park3<msg>Mv "Oh, [player_name], I wasn't expecting visitors."<|endoftext|>
[ONNX] Meet with Lorem -> loremapt<msg>Lo "Oh, [remy laptrap]! How are ya?"<p><msg>c "Oh, you're here already. I just wanted to talk to you about something."<d><scn>loremapt<msg>Lo "I see, but I'm not sure if this is the right place for you to meet."<p><msg>c "I'm not sure if I can help you with the paperwork."<d><scn>loremapt<msg>Lo "I'm sorry, but I'm not sure if this is
[ONNX Quantized] Meet with Lorem -> n<msg<v]<d><msg "<p first="<msg>c "Good to be on your side"<d><msg> "<p>"<msg>c "<fadeOut>"<msg>c "No, I can do it."<p><msg>c "And if that makes it possible for me, I will get it for free. You can even get a little of 